## Creating SQL Databases and Tables

Tables reside within a database. By default, Spark creates tables under the default database. To create your own database name, you can issue a SQL command from your Spark application or notebook. Using the US flight delays data set, let’s create both a *managed* and an *unmanaged* table. To begin, we’ll create a database called `learn_spark_db` and tell Spark we want to use that database:

In [1]:
from os.path import abspath
from pyspark.sql import SparkSession

# warehouse_location points to the default location for managed databases and tables
warehouse_location = abspath('spark-warehouse')

spark = (SparkSession
    .builder
    .appName("SparkSQLHiveExample")
    .config("spark.sql.warehouse.dir", warehouse_location)
    .enableHiveSupport()
    .getOrCreate())

22/03/01 20:17:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# Path to data set
csv_file = "../../../databricks-datasets/learning-spark-v2/flights/departuredelays.csv"

# Read and create a temporary view
# Infer schema (note that for larger files you may want to specify the schema)
df = (spark.read.format("csv")
    .option("inferSchema", "true")
    .option("header", "true")
    .load(csv_file))
df.createOrReplaceTempView("us_delay_flights_tbl")

In [3]:
# drop database and it's tables
spark.sql("drop database if exists learn_spark_db cascade")
# create database and use database
spark.sql("create database learn_spark_db")
spark.sql("use learn_spark_db")

22/03/01 20:17:47 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
22/03/01 20:17:47 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
22/03/01 20:17:49 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
22/03/01 20:17:49 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore zacks@192.168.86.250
22/03/01 20:17:49 WARN ObjectStore: Failed to get database default, returning NoSuchObjectException
22/03/01 20:17:50 WARN ObjectStore: Failed to get database learn_spark_db, returning NoSuchObjectException
22/03/01 20:17:50 WARN ObjectStore: Failed to get database learn_spark_db, returning NoSuchObjectException
22/03/01 20:17:50 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException
22/03/01 20:17:50 WARN ObjectStore: Failed to get database learn_sp

DataFrame[]

From this point, any commands we issue in our application to create tables will result in the tables being created in this database and residing under the database name `learn_spark_db`.

### Creating a managed table

To create a managed table within the database learn_spark_db, you can issue a SQL query like the following:

In [4]:
sqlQuery = """
create table if not exists managed_us_delay_flights_tbl as
select * from us_delay_flights_tbl
"""

spark.sql(sqlQuery)

22/03/01 20:17:50 WARN ResolveSessionCatalog: A Hive serde table will be created as there is no table provider specified. You can set spark.sql.legacy.createHiveTableByDefault to false so that native data source table will be created instead.
22/03/01 20:17:50 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
22/03/01 20:17:50 WARN HiveConf: HiveConf of name hive.internal.ss.authz.settings.applied.marker does not exist
22/03/01 20:17:50 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
22/03/01 20:17:50 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
22/03/01 20:17:50 WARN HiveMetaStore: Location: file:/Users/zacks/Git/Data Science Projects/LearningSparkV2/chapter4/py/src/spark-warehouse/learn_spark_db.db/managed_us_delay_flights_tbl specified for non-external table:managed_us_delay_flights_tbl


DataFrame[]

In [5]:
spark.sql("select * from managed_us_delay_flights_tbl").show()

+-------+-----+--------+------+-----------+
|   date|delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
|1121845|   -9|     198|   FSM|        DFW|
|1130620|   -8|     198|   FSM|        DFW|
|1131005|    0|     198|   FSM|        DFW|
|1131430|    0|     198|   FSM|        DFW|
|1131845|    3|     198|   FSM|        DFW|
|1140620|   -6|     198|   FSM|        DFW|
|1141005|   -7|     198|   FSM|        DFW|
|1141430|  -10|     198|   FSM|        DFW|
|1141845|  -12|     198|   FSM|        DFW|
|1150620|  -10|     198|   FSM|        DFW|
|1151005|    1|     198|   FSM|        DFW|
|1151430|   -1|     198|   FSM|        DFW|
|1151845|   -7|     198|   FSM|        DFW|
|1160620|   -5|     198|   FSM|        DFW|
|1161005|   -5|     198|   FSM|        DFW|
|1161430|   -4|     198|   FSM|        DFW|
|1161845|   -6|     198|   FSM|        DFW|
|1170620|   -1|     198|   FSM|        DFW|
|1171005|   -7|     198|   FSM|        DFW|
|1171430|   18|     198|   FSM| 

In [6]:
# drop managed_us_delay_flights_tbl we create through Spark SQL API
spark.sql("drop table if exists managed_us_delay_flights_tbl")
# Schema as defined in the preceding example
schema="date STRING, delay INT, distance INT, origin STRING, destination STRING"
flights_df = spark.read.csv(csv_file, schema=schema)
flights_df.write.saveAsTable("managed_us_delay_flights_tbl")

### Creating an unmanaged table

By contrast, you can create unmanaged tables from your own data sources—say, Parquet, CSV, or JSON files stored in a file store accessible to your Spark application.

To create an unmanaged table from a data source such as a CSV file, in SQL use:

In [7]:
sqlQuery = f"""
CREATE TABLE us_delay_flights_tbl(date STRING, delay INT, distance INT, origin STRING, destination STRING)
USING csv OPTIONS (PATH "{csv_file}")
"""

spark.sql(sqlQuery)

22/03/01 20:17:56 WARN HiveExternalCatalog: Couldn't find corresponding Hive SerDe for data source provider csv. Persisting data source table `learn_spark_db`.`us_delay_flights_tbl` into Hive metastore in Spark SQL specific format, which is NOT compatible with Hive.


DataFrame[]

And within the DataFrame API use:

In [10]:
# drop table we created in previous steps
spark.sql("drop table if exists us_delay_flights_tbl")

22/03/01 20:18:20 WARN HadoopFSUtils: The directory file:/Users/zacks/Git/Data Science Projects/LearningSparkV2/chapter4/py/databricks-datasets/learning-spark-v2/flights/departuredelays.csv was not found. Was it deleted very recently?


DataFrame[]

In [11]:
# you may also need to remove /tmp/data/us_flights_delay at first
# rm -rf /tmp/data
(flights_df
    .write
    .option("path", "/tmp/data/us_flights_delay")
    .saveAsTable("us_delay_flights_tbl"))

In [12]:
spark.sql("select * from us_delay_flights_tbl").show()

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01201755|    0|     449|   ORF|        ATL|
|01201610|   52|     449|   ORF|        ATL|
|01201441|    0|     449|   ORF|        ATL|
|01211755|  -15|     449|   ORF|        ATL|
|01210941|   -5|     449|   ORF|        ATL|
|01210700|    5|     449|   ORF|        ATL|
|01211243|    2|     449|   ORF|        ATL|
|01210540|   -5|     449|   ORF|        ATL|
|01211610|   27|     449|   ORF|        ATL|
|01211441|   -6|     449|   ORF|        ATL|
|01221755|   -4|     449|   ORF|        ATL|
|01220941|   -5|     449|   ORF|        ATL|
|01220700|   -2|     449|   ORF|        ATL|
|01221243|    0|     449|   ORF|        ATL|
|01220540|   -5|     449|   ORF|        ATL|
|01221610|   -5|     449|   ORF|        ATL|
|01221441|   -2|     449|   ORF|        ATL|
|01231755|   -1|     449|   ORF|        ATL|
|01230941|   -8|     449|   ORF|        ATL|
|01230700|

### Creating Views

In addition to creating tables, Spark can create views on top of existing tables. Views can be global (visible across all `SparkSessions` on a given cluster) or session-scoped (visible only to a single `SparkSession`), and they are temporary: they disappear after your Spark application terminates.

[Creating views](https://docs.databricks.com/spark/latest/spark-sql/language-manual/sql-ref-syntax-ddl-create-view.html#id1) has a similar syntax to creating tables within a database. Once you cre‐ ate a view, you can query it as you would a table. The difference between a view and a table is that views don’t actually hold the data; tables persist after your Spark applica‐ tion terminates, but views disappear.

You can create a view from an existing table using SQL. For example, if you wish to work on only the subset of the US flight delays data set with origin airports of New York (JFK) and San Francisco (SFO), the following queries will create global temporary and temporary views consisting of just that slice of the table:

In [13]:
sqlQuery = """
CREATE OR REPLACE GLOBAL TEMP VIEW us_origin_airport_SFO_global_tmp_view AS
SELECT date, delay, origin, destination
from us_delay_flights_tbl
WHERE origin = 'SFO';
"""

spark.sql(sqlQuery)

sqlQuery = """
CREATE OR REPLACE TEMP VIEW us_origin_airport_JFK_tmp_view AS
SELECT date, delay, origin, destination
from us_delay_flights_tbl
WHERE origin = 'JFK'
"""

spark.sql(sqlQuery)

DataFrame[]

Once you’ve created these views, you can issue queries against them just as you would against a table. Keep in mind that when accessing a global temporary view you must use the prefix `global_temp.<view_name>`, because Spark creates global temporary views in a global temporary database called `global_temp`. For example:

In [14]:
# query global view from global_temp
spark.sql("select * from global_temp.us_origin_airport_SFO_global_tmp_view").show(5)

+--------+-----+------+-----------+
|    date|delay|origin|destination|
+--------+-----+------+-----------+
|01011250|   55|   SFO|        JFK|
|01012230|    0|   SFO|        JFK|
|01010705|   -7|   SFO|        JFK|
|01010620|   -3|   SFO|        MIA|
|01010915|   -3|   SFO|        LAX|
+--------+-----+------+-----------+
only showing top 5 rows



By contrast, you can access the normal temporary view without the `global_temp` prefix:

In [15]:
# query temporary view from current database
spark.sql("select * from us_origin_airport_JFK_tmp_view").show(5)

+--------+-----+------+-----------+
|    date|delay|origin|destination|
+--------+-----+------+-----------+
|02010900|   -1|   JFK|        LAX|
|02011200|   -5|   JFK|        LAX|
|02011030|   -6|   JFK|        LAX|
|02011900|   -1|   JFK|        LAX|
|02011700|   -3|   JFK|        LAS|
+--------+-----+------+-----------+
only showing top 5 rows



You can accomplish the same thing with the DataFrame API as follows:

In [16]:
df_sfo = spark.sql("SELECT date, delay, origin, destination FROM us_delay_flights_tbl WHERE origin = 'SFO'")
df_jfk = spark.sql("SELECT date, delay, origin, destination FROM us_delay_flights_tbl WHERE origin = 'JFK'")
# Create a temporary and global temporary view
df_sfo.createOrReplaceGlobalTempView("us_origin_airport_SFO_global_tmp_view")
df_jfk.createOrReplaceTempView("us_origin_airport_JFK_tmp_view")

You can also query table from DataFrame API:

In [17]:
# query global view from global_temp
spark.read.table("global_temp.us_origin_airport_SFO_global_tmp_view").show(5)

+--------+-----+------+-----------+
|    date|delay|origin|destination|
+--------+-----+------+-----------+
|01011250|   55|   SFO|        JFK|
|01012230|    0|   SFO|        JFK|
|01010705|   -7|   SFO|        JFK|
|01010620|   -3|   SFO|        MIA|
|01010915|   -3|   SFO|        LAX|
+--------+-----+------+-----------+
only showing top 5 rows



In [18]:
# query temporary view from current database
spark.read.table("us_origin_airport_JFK_tmp_view").show(5)

+--------+-----+------+-----------+
|    date|delay|origin|destination|
+--------+-----+------+-----------+
|02010900|   -1|   JFK|        LAX|
|02011200|   -5|   JFK|        LAX|
|02011030|   -6|   JFK|        LAX|
|02011900|   -1|   JFK|        LAX|
|02011700|   -3|   JFK|        LAS|
+--------+-----+------+-----------+
only showing top 5 rows



To show views:

In [19]:
# List all views in global temp view database and current database
spark.sql("show views in global_temp;").show(truncate=False)

+-----------+-------------------------------------+-----------+
|namespace  |viewName                             |isTemporary|
+-----------+-------------------------------------+-----------+
|global_temp|us_origin_airport_sfo_global_tmp_view|true       |
|           |us_origin_airport_jfk_tmp_view       |true       |
+-----------+-------------------------------------+-----------+



In [20]:
# Or
# List all views in global temp view database and current database
spark.catalog.listTables(dbName="global_temp")

[Table(name='us_origin_airport_sfo_global_tmp_view', database='global_temp', description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='us_origin_airport_jfk_tmp_view', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

To drop views:

In [22]:
# drop views
spark.sql("drop view if exists us_origin_airport_sfo_global_tmp_view")
spark.sql("drop view if exists us_origin_airport_jfk_tmp_view")

DataFrame[]

In [21]:
# Or
# drop views
spark.catalog.dropGlobalTempView("us_origin_airport_sfo_global_tmp_view")
spark.catalog.dropTempView("us_origin_airport_jfk_tmp_view")